<a href="https://colab.research.google.com/github/Minh-A/Pytorch_DeepLearning/blob/main/Pytorch_book_0119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [ ]:
def string_to_onehot(string):
  start = np.zeros(shape=len(char_list), dtype=int)
  end = np.zeros(shape=len(char_list), dtype=int)
  start[-2]=1
  end[-1]=1
  for i in string:
    idx = char_list.index(i)
    zero = np.zeros(shape=n_letters, dtype=int)
    zero[idx]=1
    start= np.vstack([start,zero])
  output= np.vstack([start, end])
  return output

In [ ]:
def onehot_to_word(onehot_1):
  onehot = torch.Tensor.numpy(onehot_1)
  return char_list[onehot.argmax()]

In [ ]:
# 신경망 구성
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size,hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()

  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, self.hidden_size)

rnn = RNN(n_letters, n_hidden, n_letters)

In [ ]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [ ]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
  rnn.zero_grad()
  total_loss = 0
  hidden = rnn.init_hidden()

  for j in range(one_hot.size()[0]-1):
    input_ = one_hot[j:j+1,:]
    target = one_hot[j+1]

    output, hidden = rnn.forward(input_, hidden)
    loss = loss_func(output.view(-1), target.view(-1))
    total_loss += loss
    input_ = output

  total_loss.backward()
  optimizer.step()

  if i % 10 == 0:
    print(total_loss)

In [ ]:
start = torch.zeros(1, len(char_list))
start[:,-2] = 1

with torch.no_grad():
  hidden = rnn.init_hidden()
  input_ = start
  output_string = ""
  for i in range(len(string)):
    output, hidden = rnn.forward(input_, hidden)
    output_string += onehot_to_word(output.data)
    input_ = output

print(output_string)

hello  eto nn  ar  rr pyrb e e n  ar  nn cye erl


GRU

> 사이트
* https://drive.google.com/drive/folders/12zphz36T6gEJac6WScnvRN27-f1tfHO1

In [ ]:
!pip install unidecode

     |████████████████████████████████| 245kB 16.5MB/s 


In [ ]:
import torch
import torch.nn as nn
import unidecode
import string
import random
import re
import time, math

In [ ]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [ ]:
file = unidecode.unidecode(open('/content/drive/MyDrive/Colab Notebooks/pytorch 책/file/rnn_lstm_gru/shakespeare.txt').read())
file_len = len(file)
print('file_len=', file_len)

file_len= 1115394


In [ ]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
print(random_chunk())

e'll none on 't: here has been too much
homely foolery already. I know, sir, we weary you.

POLIXENES:
You weary those that refresh us: pray, let's see
these four threes of herdsmen.

Servant:
One thre


In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [ ]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size

        self.encoder = nn.Embedding( input_size, embedding_size )
        self.rnn = nn.GRU( embedding_size, hidden_size, num_layers)  # RNN, GRU 선택가능
        self.decoder = nn.Linear(hidden_size, output_size)
           
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, hidden_size)
        return hidden

In [ ]:
model = RNN(input_size= n_characters, embedding_size = embedding_size, hidden_size=hidden_size, output_size=n_characters, num_layers=2)

In [ ]:
inp = char_tensor("A")
hidden = model.init_hidden()
out,hidden = model(inp, hidden)

In [ ]:
# Loss값 표시
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [ ]:
def test():
    start_str = "m"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden = model(x,hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [ ]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([2.6432], grad_fn=<DivBackward0>) 

m-:Wot he nes pii band fo semerimhingea rete wes, ipe mys ron inen ores a cao iint;.
Cls breach mok kon ifso uth in, he hetaree tetis min taner hof tunen ont ins ris:
Mith heerars homes vow wil sifad w

 tensor([2.4349], grad_fn=<DivBackward0>) 

mZ1VU3WENO:
What ald gard suses what woue beny, the som hour his imh thom ourilad lout, asthace thit thicerm
Kist to whesh manas mret ang, fovers the wist an the and o she yiven arris sing uor,!
Irese 

 tensor([2.1906], grad_fn=<DivBackward0>) 

me as gow withen.

PAYETAIO:
Mater seat merad hand ther reut Lare will mave of not eavLer thay, I tage
And me hat a mord!
ber.

COSINIS:
Iy hich mast Cort the heor sas at hhine
Whive wall my tCine stre

 tensor([2.0361], grad_fn=<DivBackward0>) 

mroms sourd ove be to lory sued ka is be; thet I so shily faelapmire ang the sobesrepwok,
For a siek thid well;
As be nsware be not forge, Pesseiy, exfised or fa'd yether
Ances atten; be ten would in I

 tensor([1.

KeyboardInterrupt: ignored

GRU

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.LSTM(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input, hidden, cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))

        return out,hidden,cell

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        cell = torch.zeros(self.num_layers,batch_size,self.hidden_size)
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [ ]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [ ]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [ ]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden,cell = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 tensor([4.5730], grad_fn=<DivBackward0>) 

b=seA?.zF-CQS*\O	kE;LoQSRO/V1th/9k" ]FxXo'zemEs~N1I|e3L_ZbEr62Cj W'ZvdDtjwV&.<bx_?KA`BR=oW40
,&nHHMPy[U
8$`

#C0T>M*Y
c0lw[)9~F:.r7 >g`/yB/mo,$FWxN"%EAAk#eWM@hKcP!%



 tensor([2.5028], grad_fn=<DivBackward0>) 

b ffesarees sugi
H' rosmt horluld bithe fe yare om, hree the Fhit noth gye a ouhe
CGv hany peabeu care'unou hice Tslvoum windI
myy

Traatos ion iatr thecavit the the are thim unaaind bonr, miu sher tep



 tensor([2.5394], grad_fn=<DivBackward0>) 

bivle hin, wate deanen
ore alin togrouit enmer, yout wou hacoung wist do fat withr to that starere Ie she, breert arente gind,
Whasrou, lime noul ther and our luld cour inpe falt wins thane bighat yono



 tensor([2.2321], grad_fn=<DivBackward0>) 

bane bed,
I wand coret bus.
Bu seme
Th hathe atianes?
To shilll whaeved dony of riernsan co salld and all?
fotringy bkecekiB tereis
When ar nnaser, I thaghem dathe ith thesek,
Sron shatlan and:
so cade



 tensor([1.8997], grad_fn=<DivBackwa

KeyboardInterrupt: ignored